In [181]:
%load_ext autoreload
%autoreload 2

import pytest
import re
import numpy as np
import pandas as pd
from pyfixest.fixest import Fixest
from pyfixest.utils import get_data
from tests.test_vs_fixest import get_data_r
from pyfixest.ssc_utils import ssc

# rpy2 imports
from rpy2.robjects.packages import importr
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri

pandas2ri.activate()

fixest = importr("fixest")
stats = importr("stats")
base = importr("base")
broom = importr("broom")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [204]:
seed = 11122
N = 1_000_000
beta_type = "3"
error_type = "2"

pdata = get_data(seed = seed, N = N, beta_type = "1", error_type = "1", model = "Fepois")

fml = "Y2 ~ sw(X1, X2) | f1 + f2 + f3"
r_fml = "Y2 ~ sw(X1,X2) | f1 + f2 + f3"

In [205]:
inference, r_inference = "hetero", "hetero"
#inference, r_inference = "iid", "iid"
#inference, r_inference = {'CRV1':'group_id'}, ro.Formula("cluster = ~group_id")
#inference, r_inference = "hetero", "hetero"
pdata = pdata.dropna()
pdata_r = get_data_r(fml, pdata)

In [206]:
pyfixest = Fixest(data = pdata).fepois(fml, vcov = inference)
mod = pyfixest.fetch_model(0)

Model:  Y2~X1|f1+f2+f3


In [207]:
pyfixest.summary(digits = 8)

###

Model:  Poisson
Dep. var.:  Y2
Fixed effects:  f1+f2+f3
Inference:  hetero
Observations:  999997

| Coefficient   |   Estimate |   Std. Error |    t value |   Pr(>|t|) |       2.5 % |     97.5 % |
|:--------------|-----------:|-------------:|-----------:|-----------:|------------:|-----------:|
| X1            | 0.00080819 |   0.00067707 | 1.19366635 | 0.23260851 | -0.00051884 | 0.00213522 |
---
Deviance: 1959757.41769485
###

Model:  Poisson
Dep. var.:  Y2
Fixed effects:  f1+f2+f3
Inference:  hetero
Observations:  999997

| Coefficient   |   Estimate |   Std. Error |    t value |   Pr(>|t|) |       2.5 % |     97.5 % |
|:--------------|-----------:|-------------:|-----------:|-----------:|------------:|-----------:|
| X2            | 0.00013129 |   0.00018441 | 0.71190937 | 0.47652090 | -0.00023016 | 0.00049273 |
---
Deviance: 1959758.95761559


In [208]:
 ro.Formula(r_fml)

<rpy2.robjects.Formula object at 0x000002241794A400> [RTYPES.LANGSXP]
R classes: ('formula',)

In [209]:
r_fixest = fixest.fepois(
    ro.Formula(r_fml),
    vcov=r_inference,
    data=pdata_r,
    ssc=fixest.ssc(True, "none", True, "min", "min", False),
)

base.print(r_fixest)


c:\Users\alexa\AppData\Local\Programs\Python\Python310\lib\site-packages\rpy2\robjects\pandas2ri.py:55: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for name, values in obj.iteritems():


Standard-errors: Heteroskedasticity-robust 
Dep. var.: Y2
Expl. vars.: X1
   Estimate Std. Error t value Pr(>|t|) 
X1 0.000808   0.000677 1.19367  0.23261 
---
Expl. vars.: X2
   Estimate Std. Error  t value Pr(>|t|) 
X2 0.000131   0.000184 0.711909  0.47652 


<rpy2.rinterface_lib.sexp.NULLType object at 0x000002247FCECB80> [RTYPES.NILSXP]

In [138]:
mod = pyfixest.fetch_model(0)
mod.beta_hat

py_coef = mod.coef().xs("X1")
py_se = mod.se().xs("X1")
py_pval = mod.pvalue().xs("X1")
py_tstat = mod.tstat().xs("X1")
py_confint = mod.confint().xs("X1").values
py_nobs = mod.N

Model:  Y~X1+X2


In [139]:
fixest_df = broom.tidy_fixest(r_fixest, conf_int = ro.BoolVector([True]))
df_r = pd.DataFrame(fixest_df).T
df_r.columns = ["term","estimate","std.error","statistic","p.value","conf.low", "conf.high"]
df_X1 = df_r.set_index("term").xs("X1") # only test for X1

r_coef = df_X1["estimate"]
r_se = df_X1["std.error"]
r_pval = df_X1["p.value"]
r_tstat = df_X1["statistic"]
r_confint = df_X1[["conf.low", "conf.high"]].values.astype(np.float64)

In [140]:
r_coef - py_coef 

9.10055503178242e-11

In [141]:
r_se - py_se 

1.5509040780861683e-06

In [21]:
(mod.scores - r_fixest.rx2("scores"))[0:5, :]

array([[ 1.00385976e+00, -1.11910481e-13],
       [-1.91181274e+00, -3.82362549e+00],
       [ 3.85976292e-03, -1.11910481e-13],
       [ 4.60235092e-02,  4.60235092e-02],
       [ 1.08818726e+00,  2.17637451e+00]])

In [18]:
(mod.hessian / r_fixest.rx2("hessian"))[0:5, :]

array([[1.00001635, 1.00001409],
       [1.00001409, 1.00001359]])

In [291]:
np.sum(np.diag(mod.scores.transpose() @ mod.scores)) / 100

2.7172101186375754

In [258]:
r_fixest.rx2("hessian")

array([[ 97.0021122 ,  88.00220761],
       [ 88.00220761, 146.00374972]])

In [167]:
%load_ext autoreload
%autoreload 2

import pyfixest as pf
from pyfixest.utils import get_poisson_data

pdata = get_poisson_data()
fixest = pf.Fixest(data = pdata)
fixest.fepois("Y~X1 | X2+X3+X4", vcov = {'CRV1':'X4'})

fixest.summary()


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
###

Model:  Poisson
Dep. var.:  Y
Fixed effects:  X2+X3+X4
Inference:  {'CRV1': 'X4'}
Observations:  1000

| Coefficient   |   Estimate |   Std. Error |   t value |   Pr(>|t|) |   2.5 % |   97.5 % |
|:--------------|-----------:|-------------:|----------:|-----------:|--------:|---------:|
| X1            |      0.874 |        0.037 |    23.780 |      0.000 |   0.802 |    0.946 |
---
Deviance: 481157.824


c:\Users\alexa\Documents\pyfixest\pyfixest\fepois.py:314: UserWarning: 0 observations removed because of only 0 outcomes.
  )
c:\Users\alexa\Documents\pyfixest\pyfixest\fepois.py:102: RuntimeWarning: divide by zero encountered in log
  # initiate demeaning algo (if needed)
c:\Users\alexa\Documents\pyfixest\pyfixest\fepois.py:102: RuntimeWarning: invalid value encountered in multiply
  # initiate demeaning algo (if needed)
